In [18]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [19]:
data = np.loadtxt('../data/concrete.txt')
np.random.shuffle(data)
cutoff = int(len(data)*0.8)
X_train, y_train  = data[:cutoff, :-1], data[:cutoff, -1]
X_test, y_test = data[cutoff:, :-1], data[cutoff:, -1]

In [20]:
std_X_train = np.std(X_train, 0)
std_X_train[std_X_train == 0] = 1
mean_X_train = np.mean(X_train, 0)
X_train = (X_train - np.full(X_train.shape, mean_X_train)) / np.full(X_train.shape, std_X_train)
X_test = (X_test - np.full(X_test.shape, mean_X_train)) / np.full(X_test.shape, std_X_train)

In [21]:
# Set model params
model_params = {"learning_rate": 0.001,
                "dropout": 0.01,
                "hidden_width": 50}

In [22]:
def model_fn(features, labels, mode, params):
    '''
    features: A dict containing the features passed to the model via input_fn.
    labels: A Tensor containing the labels passed to the model via input_fn. 
            Will be empty for predict() calls, as these are the values the model will infer.
    mode: One of the following tf.estimator.ModeKeys string values indicating the context in which 
          the model_fn was invoked
          
    Return: tf.estimator.EstimatorSpec object
    '''
    ### 1. Configure the model via TensorFlow operations
    
    # Connect the first hidden layer to input layer
    # (features["x"]) with relu activation
    
    dropout_1 = tf.layers.dropout(inputs=features["x"],
                            rate=params['dropout'],
                            training=True)
    
    hidden_layer = tf.layers.dense(dropout_1, 
                                         units=params['hidden_width'],
                                         activation=tf.nn.relu)

    dropout_2 = tf.layers.dropout(inputs=hidden_layer,
                                rate=params['dropout'],
                                training=True)

    output_layer = tf.layers.dense(dropout_2, 
                                   units=1)

    predictions = tf.reshape(output_layer, [-1])
    
    # Provide an estimator spec for `ModeKeys.PREDICT`.
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions={"y": predictions})
    
    #onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    #loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    ### 2. Define the loss function for training/evaluation
    loss = tf.losses.mean_squared_error(labels, predictions)
    eval_metric_ops = {"rmse": 
                       tf.metrics.root_mean_squared_error(tf.cast(labels, tf.float64), predictions)}
    
    ### 3. Define the training operation/optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    
    
    ### 4. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)

In [23]:
# Instantiate Estimator
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


In [24]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=32,
    num_epochs=None,
    shuffle=False)

In [25]:
# Train
nn.train(input_fn=train_input_fn,steps=40*32)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp06v_ijxe/model.ckpt.
INFO:tensorflow:loss = 1573.19, step = 1
INFO:tensorflow:global_step/sec: 765.723
INFO:tensorflow:loss = 1219.06, step = 101 (0.132 sec)
INFO:tensorflow:global_step/sec: 792.037
INFO:tensorflow:loss = 1672.99, step = 201 (0.126 sec)
INFO:tensorflow:global_step/sec: 804.939
INFO:tensorflow:loss = 681.691, step = 301 (0.124 sec)
INFO:tensorflow:global_step/sec: 805.644
INFO:tensorflow:loss = 439.247, step = 401 (0.124 sec)
INFO:tensorflow:global_step/sec: 800.435
INFO:tensorflow:loss = 367.938, step = 501 (0.125 sec)
INFO:tensorflow:global_step/sec: 761.937
INFO:tensorflow:loss = 265.374, step = 601 (0.131 sec)
INFO:tensorflow:global_step/sec: 779.858
INFO:tensorflow:loss = 229.072, step = 701 (0.128 sec)
INFO:tensorflow:global_step/sec: 713.84
INFO:tensorflow:loss = 263.757, step = 801 (0.140 sec)
INFO:tensorflow:global_step/sec: 743.037
INFO:tensorflow:loss = 293.756, 

In [26]:
# Score accuracy
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

In [27]:
ev = nn.evaluate(input_fn=test_input_fn)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

INFO:tensorflow:Starting evaluation at 2017-12-05-21:41:01
INFO:tensorflow:Restoring parameters from /tmp/tmp06v_ijxe/model.ckpt-1280
INFO:tensorflow:Finished evaluation at 2017-12-05-21:41:01
INFO:tensorflow:Saving dict for global step 1280: global_step = 1280, loss = 147.715, rmse = 12.0603
Loss: 147.715
Root Mean Squared Error: 12.0603
